In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colab 실행시 필요한 다운로드

!pip install python-multipart >> /dev/null

#Fast api 
!pip install fastapi >> /dev/null
!pip install aiofiles >> /dev/null
!pip install pydantic >> /dev/null
!pip install pyngrok >> /dev/null
!pip install uvicorn >> /dev/null
!pip install requests >> /dev/null

#qr 코드
!pip install qrcode >> /dev/null

# yolov4 사용 하려면 4.4.0 버전 이상 OpenCv를 사용해야 함
!pip install --upgrade opencv-python >> /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [20]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/FastApi/FastApi/backend/')

In [21]:
%cd '/content/drive/MyDrive/Colab Notebooks/FastApi/FastApi/backend'
%ls

/content/drive/MyDrive/Colab Notebooks/FastApi/FastApi/backend
apis/  data/   main_yolo.ipynb  ocr/          requirements.txt  templates/
core/  image/  module.py        __pycache__/  static/           users/


In [22]:
from typing import List
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import shutil
from fastapi import FastAPI, UploadFile, File

from fastapi import Request
from fastapi.staticfiles import StaticFiles #new
from fastapi.templating import Jinja2Templates
from fastapi import HTTPException
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles

# 필요한 라이브러리 당겨오기 
from typing import Optional
# Pydantic을 이용해 파이썬 표준 타입으로 분문을 선언 
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# json, qrcode 라이브러리 가져오기 
import json
import qrcode

# FastAPI  및 웹 관련 라이브러리 가져오기 
# from core.config import settings
from apis.general_pages.route_homepage import general_pages_router


# 모델 2 필요한 것 
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow # colab에서는 이걸 사용해야함
from glob import glob

# json 데이터 가져오기 
with open('./users/pet_num.json', 'r') as fp:
    data = json.load(fp)


classes = []
# class의 이름들
with open("./data/custom.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]


Module 1 : 사진파일 >> 텍스트 >> id 값 가져오기

In [23]:
# module py 파일 가져오기 
from module import call_picture, yolo, pet_total, pet_sum

In [24]:
# 개인 ngrok token
auth = '21zKe7LoaZJ2O6jgteldfIw9As9_25LiPNZUoFoMMqCGRsv5E'

In [27]:
from typing import List
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import shutil
import os
import time
from fastapi import FastAPI, UploadFile, File, Form

from fastapi import Request
from fastapi.staticfiles import StaticFiles #new
from fastapi.templating import Jinja2Templates
from fastapi import HTTPException
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles

app = FastAPI()

app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# qr코드 생성하는 부분 >> 개수, 아이디 받아오는 구문 하나 만들고, 변수 활용하는 것도 포함 시키기  

# 페트병 개수, 사용자 아이디를 FAST API 로 출력하기 
def make_qr(id, num, point):
  url = f'{ngrok_tunnel.public_url}/pet/?user_id={id}&num={num}&point={point}'
  img = qrcode.make(url)
  img.save('./static/images/qr.png')
  return url

# 메인페이지 
@app.get('/')
def root(request:Request):
    return templates.TemplateResponse('general_pages/homepage.html', context={'request': request, 'url' : ngrok_tunnel.public_url+'/upload-card' })


@app.get("/upload-test")
def form_post(request: Request):
    """Displays the form to create a project"""
    return templates.TemplateResponse('upload-alerts.html', context={'request': request})

@app.get("/upload-test/file")
def form_post(request: Request):
    """Displays the form to create a project"""
    return {"message": "Upload success"}

# 파일 업로드 >> 딥러닝 결과 출력 (Pass or Fail )
@app.post("/upload-test")
async def images(request: Request,
                    files: List[UploadFile] = File(...)):
                    #file: UploadFile = File(...)):

    for image in files:
        print(image)
        with open(f"users/images/{image.filename}", 'wb') as buffer:
            shutil.copyfileobj(image.file, buffer)
    
    # 사용자 정보 
    card = './ocr/pet_card2.jpg'
    user_id = call_picture(card)

    # 페트병 개수 
    image_list = glob('./users/images/*.jpg')
    pet_num = pet_total(image_list)
    
    # 결과
    num, point = pet_sum(user_id, pet_num)

    total = len(files)
    pet_t = total
    pass_t = pet_num
    
    # 업로드 이미지 삭제
    for img_path in image_list:
      os.remove(img_path)

    # 예측된 이미지 경로 리스트
    pre_img = glob('./static/images/*.jpg')
    
    # html내에서 url_for 로 불러올때 경로의 부딪침을 방지하기 위해 경로에서 ./static을 제거
    for i, p_path in enumerate(pre_img):
      pre_img[i] = p_path[8:]
    print(f'pre_img : {pre_img}')

    # qr코드 만들기
    url = make_qr(user_id, num, point)

    # 그 결과를 출력하는 웹 html
    return templates.TemplateResponse("p_test.html", {"request": request, 'total':total, 'pet_t':pet_t, 'pass_t':pass_t, 'url':url, 'img_path': pre_img})
    

# 결과 출력 창 (딕셔너리에서 결과값만 출력하도록 설정) >> 고유한 숫자로 fix 될 수 있도록 
# 템플릿 디자인 수정 했지만, 필요시 추가 수정 필요 
@app.get('/pet/', response_class=HTMLResponse)
async def pet(user_id:str, num:int, point:int, request: Request):

  # 예측한 이미지 삭제
  image_list = glob('./static/images/*.jpg')
  for img_path in image_list:
    os.remove(img_path)
    
  time.sleep(1)
  return templates.TemplateResponse("main.html", {"request": request, 'id':user_id, 'pet_num':num, 'point':point})


# OCR 시연 
@app.get("/upload-card")
def test_ocr(request: Request):
    """Displays the form to create a project"""
    return templates.TemplateResponse('upload-ocr.html', context={'request': request})

@app.post("/upload-card")
async def ocr_result(request: Request,
                    file: UploadFile = File(...)
                    ):
  # jpg -> png로 변경, 이미지를 뿌릴때 jpg만 glob에서 뿌리기 위해 card 이미지는 .png로 저장
    with open("static/images/card.png", 'wb') as buffer:
        shutil.copyfileobj(file.file, buffer)
    
    user_id = call_picture("static/images/card.png")

    return templates.TemplateResponse("result-ocr.html", {"request": request, 'user_id':user_id, 'url': ngrok_tunnel.public_url+'/upload-test'})

ngrok.set_auth_token(auth)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://85ca-35-197-82-230.ngrok.io


INFO:     Started server process [74]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     122.35.78.141:0 - "GET / HTTP/1.1" 200 OK
INFO:     122.35.78.141:0 - "GET /static/images/logo.png HTTP/1.1" 200 OK
INFO:     122.35.78.141:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     122.35.78.141:0 - "GET /upload-card HTTP/1.1" 200 OK
INFO:     122.35.78.141:0 - "POST /upload-card HTTP/1.1" 200 OK
INFO:     122.35.78.141:0 - "GET /static/images/card.png HTTP/1.1" 200 OK
INFO:     122.35.78.141:0 - "GET /upload-test HTTP/1.1" 200 OK
[clean(3)] conf: 0.48405835032463074 / x: 85 / y: 15 / width: 56 / height: 125
[label(5)] conf: 0.9624748229980469 / x: 42 / y: 41 / width: 94 / height: 79
[label(6)] conf: 0.97287517786026 / x: 46 / y: 28 / width: 114 / height: 36
[label(5)] conf: 0.8828713893890381 / x: 34 / y: 55 / width: 138 / height: 50
[label(3)] conf: 0.8566361665725708 / x: 49 / y: 34 / width: 116 / height: 70
[clean(2)] conf: 0.5688760280609131 / x: 50 / y: 16 / width: 50 / height: 101
총 10의 페트병 중 2개의 페트병이 통과되었고, 4개의 페트병 이미지는 인식하지 못했습니다.
pre_img : ['/images/p

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74]


### ngrok 왜 날 괴롭혀!

In [14]:
# ngrok에 authtoken 설정
!ngrok authtoken 21zKe7LoaZJ2O6jgteldfIw9As9_25LiPNZUoFoMMqCGRsv5E

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!ngrok help

In [ ]:
!ngrok start --none

### 이미지 크기 조정

In [ ]:
from google.colab.patches import cv2_imshow

src = cv2.imread("/content/drive/MyDrive/Colab Notebooks/FastApi/FastApi/backend/image/f/ff/KakaoTalk_20211117_002238726_00.jpg", cv2.IMREAD_COLOR)

dst = cv2.resize(src, dsize=(256, 256), interpolation=cv2.INTER_AREA)
dst2 = cv2.resize(src, dsize=(0, 0), fx=0.1, fy=0.1, interpolation=cv2.INTER_LINEAR)